**Fine-Tuning Language Models for PII Identification and Replacement**


In [1]:
!pip install -q transformers[torch] datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 6.5 MB/s eta 0:00:00


In [3]:
import torch
from transformers import AutoTokenizer, Qwen2ForCausalLM,Qwen2TokenizerFast
from datasets import load_dataset
from trl import SFTTrainer
from peft import LoraConfig
from transformers import TrainingArguments


In [ ]:
HF_TOKEN = '<paste your HF token here>'

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:

model = Qwen2ForCausalLM.from_pretrained("Qwen/Qwen1.5-0.5B",token=HF_TOKEN).to(device)
tokenizer = Qwen2TokenizerFast.from_pretrained("Qwen/Qwen1.5-0.5B",token=HF_TOKEN)
prompt = "A student's assessment was found on device bearing IMEI: 06-184755-866851-3. The document falls under the various topics discussed in our Optimization curriculum. Can you please collect it?"
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# Generate
generate_ids = model.generate(inputs.input_ids)
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


"A student's assessment was found on device bearing IMEI: 06-184755-866851-3. The document falls under the various topics discussed in our Optimization curriculum. Can you please collect it? Please note that the IMEI is a unique identifier for the device, and it is not a valid IMEI. It is possible that the device is not registered with the IMEI, or it may be a device that has been stolen or has been modified. In such cases, it is important to contact the device manufacturer or the device owner for assistance.\n\nCan you please provide me with the IMEI of the device that was found on the student's device? I'm sorry, but as an AI language model, I don't have access to the IMEI of the device that was found on the student's device. The IMEI is a unique identifier for the device, and it is not a valid IMEI. It is possible that the device is not registered with the IMEI, or it may be a device that has been stolen or has been modified. In such cases, it is important to contact the device manu

**Dataset** 
Masking personally identifiable information dataset

In [ ]:
dataset = load_dataset("ai4privacy/pii-masking-200k") #source https://huggingface.co/datasets/ai4privacy/pii-masking-200k


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/12.8k [00:00<?, ?B/s]

english_pii_43k.jsonl:   0%|          | 0.00/73.8M [00:00<?, ?B/s]

french_pii_62k.jsonl:   0%|          | 0.00/116M [00:00<?, ?B/s]

german_pii_52k.jsonl:   0%|          | 0.00/97.8M [00:00<?, ?B/s]

italian_pii_50k.jsonl:   0%|          | 0.00/93.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/209261 [00:00<?, ? examples/s]

The dataset contains various splits, each with a certain number of rows. In our case, as we're going to do supervised fine-tuning (SFT), only the "train_sft" and "test_sft" splits are relevant for us.

understanding the dataset 

In [7]:
sample = dataset["train"][0]
sample.keys()

dict_keys(['source_text', 'target_text', 'privacy_mask', 'span_labels', 'mbert_text_tokens', 'mbert_bio_labels', 'id', 'language', 'set'])

In [8]:
def tokenize_function(examples):
    source = tokenizer(examples["source_text"], truncation=True, padding="max_length", max_length=512)
    target = tokenizer(examples["target_text"], truncation=True, padding="max_length", max_length=512)
    return {"input_ids": source["input_ids"], "attention_mask": source["attention_mask"], "labels": target["input_ids"]}

tokenized_dataset = dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/209261 [00:00<?, ? examples/s]

In [ ]:
# Select the first 10,000 samples
small_dataset = tokenized_dataset["train"].select(range(10000))


Let's check some examples. The important thing is that each example should contain strings of `source_text` and `target_text` and privacy_mask :

In [ ]:
for i in range(3):  # example for displaying first few sampples of data
    print(small_dataset[i])

{'source_text': "A student's assessment was found on device bearing IMEI: 06-184755-866851-3. The document falls under the various topics discussed in our Optimization curriculum. Can you please collect it?", 'target_text': "A student's assessment was found on device bearing IMEI: [PHONEIMEI]. The document falls under the various topics discussed in our [JOBAREA] curriculum. Can you please collect it?", 'privacy_mask': [{'value': '06-184755-866851-3', 'start': 57, 'end': 75, 'label': 'PHONEIMEI'}, {'value': 'Optimization', 'start': 138, 'end': 150, 'label': 'JOBAREA'}], 'span_labels': '[[0, 57, "O"], [57, 75, "PHONEIMEI"], [75, 138, "O"], [138, 150, "JOBAREA"], [150, 189, "O"]]', 'mbert_text_tokens': ['A', 'student', "'", 's', 'assessment', 'was', 'found', 'on', 'device', 'bearing', 'IM', '##E', '##I', ':', '06', '-', '1847', '##55', '-', '866', '##85', '##1', '-', '3', '.', 'The', 'document', 'falls', 'under', 'the', 'various', 'topics', 'discussed', 'in', 'our', 'Op', '##timi', '##za


As we can observe in 'target_text' in the above samples the personal information is replaced by corresponding place holder tokens 
we need to add these special tokens to the tokenizer as they are not present previously

In [ ]:

special_tokens_dict = {
    "additional_special_tokens": [
    "[PREFIX]",
    "[FIRSTNAME]",
    "[LASTNAME]",
    "[DATE]",
    "[TIME]",
    "[PHONEIMEI]",
    "[USERNAME]",
    "[CITY]",
    "[STATE]",
    "[URL]",
    "[JOBAREA]",
    "[EMAIL]",
    "[JOBTITLE]",
    "[COMPANYNAME]",
    "[STREET]",
    "[COUNTY]",
    "[AGE]",
    "[USERAGENT]",
    "[ACCOUNTNAME]",
    "[ACCOUNTNUMBER]",
    "[CURRENCYSYMBOL]",
    "[AMOUNT]",
    "[CREDITCARDISSUER]",
    "[CREDITCARDNUMBER]",
    "[PHONENUMBER]",
    "[SEX]",
    "[DOB]",
    "[PIN]",
    "[IP]",
    "[CURRENCY]",
    "[PASSWORD]",
    "[CURRENCYNAME]",
    "[LITECOINADDRESS]",
    "[BUILDINGNUMBER]",
    "[ORDINALDIRECTION]",
    "[MASKEDNUMBER]",
    "[ZIPCODE]",
    "[BIC]",
    "[IPV4]",
    "[IPV6]",
    "[MAC]",
    "[NEARBYGPSCOORDINATE]",
    "[VEHICLELIN]",
    "[EYECOLOR]",
    "[HEIGHT]",
    "[SSN]"
]
}

In [12]:
tokenizer.add_special_tokens(special_tokens_dict)
model.resize_token_embeddings(len(tokenizer))  # Resize model embeddings for new tokens

Embedding(151692, 1024)

**Preparing Arguments for training**

In [ ]:

# path where the Trainer will save its checkpoints and logs
output_dir = '/content/hugging_faces'

# based on config
training_args = TrainingArguments(
    fp16=True, # specify bf16=True instead when training on GPUs that support bf16
    do_eval=False,
    evaluation_strategy='no',
    gradient_accumulation_steps=128,
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},
    learning_rate=2.0e-05,
    log_level="info",
    logging_steps=5,
    logging_strategy="steps",
    lr_scheduler_type="cosine",
    max_steps=-1,
    num_train_epochs=1,
    output_dir=output_dir,
    overwrite_output_dir=True,
    per_device_eval_batch_size=1, # originally set to 8
    per_device_train_batch_size=1, # originally set to 8
    report_to='none',
    save_strategy="no",
    save_total_limit=None,
    seed=42,
)

# based on config
peft_config = LoraConfig(
        r=64,
        lora_alpha=16,
        lora_dropout=0.1,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
)

trainer = SFTTrainer(
        model=model,
        args=training_args,
        train_dataset=small_dataset,
        tokenizer=tokenizer,
        peft_config=peft_config,
    )

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-13-fe66c73e96b5>:39: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Converting train dataset to ChatML:   0%|          | 0/10000 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/10000 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/10000 [00:00<?, ? examples/s]

Using auto half precision backend


In [14]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: language, set, privacy_mask, mbert_bio_labels, source_text, target_text, span_labels, mbert_text_tokens, id. If language, set, privacy_mask, mbert_bio_labels, source_text, target_text, span_labels, mbert_text_tokens, id are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 10,000
  Num Epochs = 1
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 128
  Total optimization steps = 78
  Number of trainable parameters = 12,582,912
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
5,3.996900
10,3.919600
15,3.916400
20,3.926200
25,3.880000
30,3.850000
35,3.928500
40,3.920800
45,3.853200
50,3.842000




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=78, training_loss=3.8805511058905187, metrics={'train_runtime': 3187.1778, 'train_samples_per_second': 3.138, 'train_steps_per_second': 0.024, 'total_flos': 9844979065159680.0, 'train_loss': 3.8805511058905187})

In [ ]:
trainer.save_model('lora-adapter') #save lora weights locally 


Saving model checkpoint to lora-adapter
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--Qwen--Qwen1.5-0.5B/snapshots/8f445e3628f3500ee69f24e1303c9f10f5342a39/config.json
Model config Qwen2Config {
  "architectures": [
    "Qwen2ForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151643,
  "hidden_act": "silu",
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 2816,
  "max_position_embeddings": 32768,
  "max_window_layers": 21,
  "model_type": "qwen2",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "num_key_value_heads": 16,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 1000000.0,
  "sliding_window": null,
  "tie_word_embeddings": true,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.48.3",
  "use_cache": true,
  "use_sliding_window": false,
  "vocab_size": 151936
}

/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:260: UserW

In [ ]:
trainer.push_to_hub("Gowtham122/qwenloraadapter", token=HF_TOKEN)


In [ ]:

# # Push to the Hub (repository will be created automatically if it doesn't exist)
# model.push_to_hub("Gowtham122/qwenloraadapter", token=HF_TOKEN)
# tokenizer.push_to_hub("Gowtham122/qwenloraadapter", token=HF_TOKEN)

Configuration saved in /tmp/tmppibpxgyc/config.json
Configuration saved in /tmp/tmppibpxgyc/generation_config.json
Model weights saved in /tmp/tmppibpxgyc/model.safetensors
Uploading the following files to Gowtham122/qwenloraadapter: generation_config.json,model.safetensors,README.md,config.json


model.safetensors:   0%|          | 0.00/1.91G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

tokenizer config file saved in /tmp/tmpx9ouc5zr/tokenizer_config.json
Special tokens file saved in /tmp/tmpx9ouc5zr/special_tokens_map.json
Uploading the following files to Gowtham122/qwenloraadapter: tokenizer.json,merges.txt,added_tokens.json,special_tokens_map.json,vocab.json,tokenizer_config.json,README.md


tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Gowtham122/qwenloraadapter/commit/eb1201bbddc32244460864db141f4df5a69f3751', commit_message='Upload tokenizer', commit_description='', oid='eb1201bbddc32244460864db141f4df5a69f3751', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Gowtham122/qwenloraadapter', endpoint='https://huggingface.co', repo_type='model', repo_id='Gowtham122/qwenloraadapter'), pr_revision=None, pr_num=None)

In [ ]:
from huggingface_hub import scan_cache_dir

# Scan the cache and delete everything so to load the new peft model
cache_dir = scan_cache_dir()
cache_dir.delete()

print("Entire Hugging Face cache cleared.")

In [ ]:
base_model = Qwen2ForCausalLM.from_pretrained("Qwen/Qwen1.5-0.5B",token=HF_TOKEN).to(device)
base_tokenizer = Qwen2TokenizerFast.from_pretrained("Qwen/Qwen1.5-0.5B",token=HF_TOKEN)

In [ ]:
#Aditionallly at inference time along with loading the model we have to add special tokens information as well before loading the lora weights 
special_tokens_dict = {
    "additional_special_tokens": [
    "[PREFIX]",
    "[FIRSTNAME]",
    "[LASTNAME]",
    "[DATE]",
    "[TIME]",
    "[PHONEIMEI]",
    "[USERNAME]",
    "[CITY]",
    "[STATE]",
    "[URL]",
    "[JOBAREA]",
    "[EMAIL]",
    "[JOBTITLE]",
    "[COMPANYNAME]",
    "[STREET]",
    "[COUNTY]",
    "[AGE]",
    "[USERAGENT]",
    "[ACCOUNTNAME]",
    "[ACCOUNTNUMBER]",
    "[CURRENCYSYMBOL]",
    "[AMOUNT]",
    "[CREDITCARDISSUER]",
    "[CREDITCARDNUMBER]",
    "[PHONENUMBER]",
    "[SEX]",
    "[DOB]",
    "[PIN]",
    "[IP]",
    "[CURRENCY]",
    "[PASSWORD]",
    "[CURRENCYNAME]",
    "[LITECOINADDRESS]",
    "[BUILDINGNUMBER]",
    "[ORDINALDIRECTION]",
    "[MASKEDNUMBER]",
    "[ZIPCODE]",
    "[BIC]",
    "[IPV4]",
    "[IPV6]",
    "[MAC]",
    "[NEARBYGPSCOORDINATE]",
    "[VEHICLELIN]",
    "[EYECOLOR]",
    "[HEIGHT]",
    "[SSN]"
]
}

In [ ]:
base_tokenizer.add_special_tokens(special_tokens_dict)
base_model.resize_token_embeddings(len(tokenizer))

Embedding(151692, 1024)

In [8]:
from peft import PeftModel, LoraConfig, get_peft_model

In [ ]:
peftmodel = PeftModel.from_pretrained(base_model, '/content/lora-adapter').to(device)

In [ ]:
prompts ="A student's assessment was found on device bearing IMEI: 06-184755-866851-3. The document falls under the various topics discussed in our Optimization curriculum. Can you please collect it?"
inputs = base_tokenizer(prompts, return_tensors="pt").to(device)

# Generate
generate_ids = peftmodel.generate(inputs.input_ids)
base_tokenizer.batch_decode(generate_ids, skip_special_tokens=False, clean_up_tokenization_spaces=False)[0]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


"A student's assessment was found on device bearing IMEI: 06-184755-866851-3. The document falls under the various topics discussed in our Optimization curriculum. Can you please collect it? Please note that the IMEI is a unique identifier for the device, and it is not a valid device identifier. It is possible that the device is not registered with the IMEI, or it may be a device that has been deleted or lost. Please provide more information about the device, including its model number, serial number, and any other relevant details, so that we can assist you better.<|endoftext|>"